Step 1:
Load classes and weather data

Run the cell to:
-load the SQLContext, 
-create instance of SQLContext and
-read the weather data into a DataFrame

In [1]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

Step 2:
Print the summary statistics for ALL COLUMNS using describe()

In [3]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
number,1095,547.0,316.24357700987383,0,1094
air_pressure_9am,1092,918.8825513138097,3.1841611803868353,907.9900000000024,929.3200000000012
air_temp_9am,1090,64.93300141287075,11.175514003175877,36.752000000000685,98.90599999999992
avg_wind_direction_9am,1091,142.23551070057584,69.13785928889183,15.500000000000046,343.4
avg_wind_speed_9am,1092,5.50828424225493,4.552813465531715,0.69345139999974,23.554978199999763
max_wind_direction_9am,1092,148.9535179651692,67.23801294602951,28.89999999999991,312.19999999999993
max_wind_speed_9am,1091,7.019513529175272,5.59820917078096,1.1855782000000479,29.84077959999996
rain_accumulation_9am,1089,0.20307895225211126,1.5939521253574904,0.0,24.01999999999907
rain_duration_9am,1092,294.1080522756142,1598.078778660148,0.0,17704.0


Let's just look at the statistics for the air temp at 9am:

In [6]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287075|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



Notice that the total number of rows in the DataFrame is 1095 while the number of rows in air_temp_9am is 1090.

This indicates that there are missing values in air_temp_9am

The following command shows the total number of rows in the DataFrame:

In [7]:
df.count()

1095

Step 3:
Remove missing values

We can drop all the rows missing a value in any calling using na.drop()

In [8]:
removeAllDF = df.na.drop()

Let's look at the summary statistics for air_temp_9am with the MISSING VALUE DROPPED:

In [9]:
removeAllDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558739|
| stddev|11.168033449415699|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



Now we have to compare the summary statistics before and after dropping the missing value

We can see that the mean and standard deviation is close to the original values, for mean is 64.933 vs. 65.022 and standard deviation is 11.175 vs. 11.168

Also you can notice that the count is 1064, which means that 1095-1064 = 31 rows dropped.

We can see this agrees with the total number of rows in the new DataFrame:

In [10]:
removeAllDF.count()

1064

By far the process goes like:

Read the file into DataFrame -> 
Print summary statistics ->
Look at statistics for air_temp_9am ->
Remove missing values ->
Look at statistics for air_temp_9am after dropping->
Look at number of rows for DataFrame

Step 4:
Impute missing values

Instead of removing rows containing missing values,
let's replace the values with the mean value for that column.

FIRST, we'll load the avg function and make a COPY of the original DataFrame for future comparision

In [11]:
from pyspark.sql.functions import avg
imputeDF = df

Next, we'll iterate through each column in the DataFrame:

compute the mean value fot that COLUMN and replace any MISSING VALUES in that column with the mean.

In [12]:
for x in imputeDF.columns:
    meanValue = removeAllDF.agg(avg(x)).first()[0]
    print(x, meanValue)
    imputeDF = imputeDF.na.fill(meanValue, [x])

number 545.0018796992481
air_pressure_9am 918.9031798641055
air_temp_9am 65.02260949558739
avg_wind_direction_9am 142.30675564934032
avg_wind_speed_9am 5.485793050713691
max_wind_direction_9am 148.48042413321312
max_wind_speed_9am 6.9997136588756925
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996038
relative_humidity_9am 34.07743985327712
relative_humidity_3pm 35.14838093290537


The agg() function performs an aggregate calculation on the DataFrame and avg(x) specifies to compute the mean on column x.

The agg() function returns a DataFrame, first() returns the FIRST row, and [0] gets the first value.

The LAST line of the code uses na.fill() to replace the missing values with the mean value(FIRST ARGUEMENT) in column x(SECOND ARGUEMENT).

The output of executing this cell prints the mean values for each column and we can see the mean value for air_temp_9am is the same mean when we remove all the missing values in step 4, i.e., 65.022

# Either we remove the missing value or replace them with mean, we get the same result

STEP 5:
Print imputed data summary statistics

Let's call describe() to show the summary statistics for the ORIGINAL and IMPUTED air_temp_9am

In [13]:
df.describe(['air_temp_9am']).show()
imputeDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287075|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219822|
| stddev|11.149948199920226|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

